In [5]:
import pandas as pd
from datetime import datetime
from sklearn.tree import DecisionTreeClassifier

### **Importa os datasets**

In [6]:
treino = pd.read_csv('./datasets/train.csv')
teste = pd.read_csv('./datasets/test.csv')
treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### **Organiza os dados**

In [7]:
# Remove colunas aparentemente irrelevantes para o modelo
col_remove = ['Name', 'Ticket', 'Cabin']
treino = treino.drop(col_remove, axis=1)
teste = teste.drop(col_remove, axis=1)

treino.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


### **Missing Values: identificando e ajustando**

In [8]:
na = pd.DataFrame()
na['treino'] = (treino.isna().sum() / len(treino.index) * 100)
na['teste'] = (teste.isna().sum() / len(teste.index) * 100)
na.sort_values(['treino','teste'] ,ascending=False)

,treino,teste
Age,19.865320,20.574163
Embarked,0.224467,0.000000
Fare,0.000000,0.239234
PassengerId,0.000000,0.000000
Pclass,0.000000,0.000000
Sex,0.000000,0.000000
SibSp,0.000000,0.000000
Parch,0.000000,0.000000
Survived,0.000000,NaN


In [9]:
treino['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [10]:
treino['Age'].fillna(treino['Age'].mean(), inplace=True)  #Média das idades
treino['Embarked'].fillna(treino['Embarked'].value_counts().index[0], inplace=True)  #Maior ocorrência(S)

teste['Age'].fillna(teste['Age'].mean(), inplace=True)
teste['Fare'].fillna(teste['Fare'].mean(), inplace=True)

na['treino'] = (treino.isna().sum() / len(treino.index) * 100)
na['teste'] = (teste.isna().sum() / len(teste.index) * 100)
na

,treino,teste
PassengerId,0.0,0.0
Survived,0.0,NaN
Pclass,0.0,0.0
Sex,0.0,0.0
Age,0.0,0.0
SibSp,0.0,0.0
Parch,0.0,0.0
Fare,0.0,0.0
Embarked,0.0,0.0


In [11]:
treino['Embarked'].value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

### **Trata dados categorizados**

In [12]:
treino = pd.get_dummies(treino)
teste = pd.get_dummies(teste)
treino.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,2,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,3,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,4,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,5,0,3,35.0,0,0,8.0500,0,1,0,0,1


### **Separa "features" e "target"**

In [13]:
X_treino = treino.drop('Survived', axis=1)
y_treino = treino['Survived']
X_teste = teste

### **Define o modelo, treina e avalia o resultado**

In [14]:
modelo = DecisionTreeClassifier(max_depth=3, random_state=42)
modelo.fit(X_treino, y_treino)
modelo.score(X_treino, y_treino)

0.8271604938271605

In [15]:
feat_imp = modelo.feature_importances_
feat = pd.DataFrame(list(zip(X_treino,feat_imp)))
feat.columns = ['Feature','Importance']
feat.set_index('Feature', inplace=True)
feat.sort_values('Importance', ascending=False)

,Importance
Feature,
Sex_female,0.628880
Pclass,0.213525
Age,0.061867
Fare,0.050802
SibSp,0.044927
PassengerId,0.000000
Parch,0.000000
Sex_male,0.000000
Embarked_C,0.000000


### **Predição**

In [16]:
y_teste = modelo.predict(X_teste)
y_teste

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,

### **Resultado para submissão**

In [17]:
resultado = pd.DataFrame()
resultado['PassengerId'] = X_teste['PassengerId']
resultado['Survived'] = y_teste
resultado.set_index('PassengerId', inplace=True)
resultado.head()

,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1


In [18]:
t = datetime.now().strftime('%Y%m%d_%H%M')
resultado.to_csv(f'./submissions/titanic_{t}.csv')